<a href="https://colab.research.google.com/github/gremoni/PCS5787_Grupo_5/blob/main/prepara_agregado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importa os pacotes necessários
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# carrega a base de dados agregada do Zenodo
df = pd.read_csv('https://zenodo.org/record/5774093/files/Agragado.csv?download=1')

In [3]:
#exclusão das colunas que apresentam apenas dados do hardware da estação (tensão da bateria e temperatura do CPU) e da coluna de Pressão Atmosférica Reduzida ao Nível do Mar, por apresentar uma grande quantidade de observações ausentes
df = df.drop(['PRESSAOATMOSFERICAREDUZIDANIVELDOMAR', 'TENSAODABATERIADAESTACAO', 'TEMPERATURADACPUDAESTACAO'], 1)

In [4]:
#exclusão das linhas que possuem dados faltantes
df = df.dropna()

In [5]:
# cria uma nova coluna contendo o dado binário de ocorrência (1) ou não (0) de focos de incêndio
def label_focos (row):
   if row['Focos'] == 0 :
      return 0
   return 1

df['Focos_bin'] = df.apply(lambda row: label_focos(row), axis=1)

In [6]:
#cria o dataframe "sim" apenas com as observações binárias de foco iguais a 1
sim = df[df['Focos_bin'] == 1]

#cria o dataframe "não" apenas com as observações binárias de foco iguais a 0
nao = df[df['Focos_bin'] == 0]

In [7]:
#cria o dataframe "nao_bal", selecionando do dataframe "nao" de maneira aleatória a mesma quantidade de linhas existentes no dataframe "sim" 
nao_bal = nao.sample(n = len(sim))

In [8]:
#cria o dataframe balanceado "df_bal", com a mesma quantidade de observações cujo atributo binário é 1 e 0 
df_bal = pd.concat([sim, nao_bal])

In [9]:
# cria o dataframe pronto para a modelegam envolvendo algoritmos de classificação binária, removendo as colunas de data, hora e número de focos 
dff = df_bal.drop(['Data', 'Hora', 'Focos'], 1)

In [10]:
# separa o dataframe dff em dois dataframes: o X contendo os atributos para treinamento dos modelos e o y com a variável target ("Focos_bin")
X = dff.loc[:, dff.columns != 'Focos_bin']
y = dff.loc[:, dff.columns == 'Focos_bin']

In [11]:
# normaliza as variáveis de treinamento
X_norm = preprocessing.normalize(X)
X_norm_df = pd.DataFrame(X_norm, columns=X.columns)

In [12]:
# define as amostras de treino (88%) e de teste (12%)
X_train, X_test, y_train, y_test = train_test_split(
X_norm_df, y, train_size=0.88)